# Tutorial for `skpomade`:  PrObabilistic MAtrix DEcompositions

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.sparse.linalg import aslinearoperator

In [ ]:
from skpomade.utils import build_test_matrix
from skpomade.range_approximation import randomized_range_finder, adaptive_randomized_range_finder

In [ ]:
# Test behavior of build_test_matrix
m = 29
n = 17
seed = 0
p_max = 5
rand_state = np.random.RandomState(seed)
A = rand_state.randn(m, n)
A = A / np.linalg.norm(A, ord=2)
M = A.copy()
for p in range(p_max):
    U, s, Vh = np.linalg.svd(M)
    plt.semilogy(s, label=r'$(A A^T)^{} \times A$'.format(p))
    M = A @ A.T @ M
plt.title(r'Singular values of $(A A^T)^p \times A$')
plt.legend()
plt.grid()

plt.figure()
for p in range(p_max):
    M = build_test_matrix(m, n, p=p, rand_state=seed)
    U, s, Vh = np.linalg.svd(M)
    plt.semilogy(s, label='p={}'.format(p))
plt.title('Similar result using build_test_matrix(m, n, p)')
plt.legend()
plt.grid()


In [ ]:
# Approximation quality of randomized_range_finder(A, n_l=l)
m = 29
n = 23
p = 3
A = build_test_matrix(m, n, p=10, rand_state=0)
_, s, _ = np.linalg.svd(A)
for k in range(1, 18):
    Q = randomized_range_finder(A, n_l=k+p)
    print("k =", k, "p =", p, 
          "||A - Q @ Q.T @ A|| < s[k]?", np.linalg.norm(A - Q @ Q.T @ A, ord=2) < s[k], 
          "({} < {})".format(np.linalg.norm(A - Q @ Q.T @ A, ord=2), s[k]))

In [ ]:
m = 290
n = 170
A = build_test_matrix(m, n, p=10, rand_state=0)
_, s, _ = np.linalg.svd(A)
plt.figure(figsize=(15,5))
plt.semilogy(s)
plt.grid()
for l in range(10):
    tolerance = 10**-l
    Q = adaptive_randomized_range_finder(A, tolerance=tolerance, r=5)
    print(tolerance, np.linalg.norm(A-Q@Q.T@A, ord=2), Q.shape[1])

In [ ]:
m = n = 345
A = build_test_matrix(m, n, p=10, rand_state=0)
Aop = aslinearoperator(A)
tolerance = 10**-5
%timeit -r2 -p2 Q = adaptive_randomized_range_finder(Aop, tolerance=tolerance, r=5)
A_rank = Q.shape[1]
print(A_rank)
%timeit -r2 -p2 QQ = randomized_range_finder(Aop, n_l=A_rank)
QQ = randomized_range_finder(Aop, n_l=A_rank)
%timeit -r2 -p2 np.linalg.svd(np.conjugate(QQ).T @ A)
%timeit -r2 -p2 ss=sp.sparse.linalg.svds(Aop, k=A_rank)
%timeit -r2 -p2 sss=np.linalg.svd(A)